In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Mostrar solo errores

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from dask_ml.model_selection import GridSearchCV
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import dask.array as da

In [3]:
# Configurar Dask para usar GPUs
cluster = LocalCUDACluster()
client = Client(cluster)

2024-07-17 02:36:25,068 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-bax05fw_', purging
2024-07-17 02:36:25,069 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2024-07-17 02:36:25,069 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [4]:
# Definir una función para crear el modelo Keras
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [5]:
# Generar datos de ejemplo binarios
X = da.random.random((1000, 8), chunks=(100, 8))
y = da.random.randint(0, 2, size=(1000,), chunks=(100,))

In [6]:
# Crear el clasificador Keras
model = KerasClassifier(model=create_model, epochs=10, batch_size=32, verbose=0)

In [7]:
# Configurar GridSearchCV con Dask
param_grid = {
    'epochs': [10, 20],
    'batch_size': [32, 64]
}
grid = GridSearchCV(model, param_grid, cv=3)
grid.fit(X, y)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1721183788.670866   57017 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721183788.742264   57017 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721183788.744293   57017 cuda_executor.cc:1015] successf

GridSearchCV(cv=3,
             estimator=KerasClassifier(batch_size=32, epochs=10, model=<function create_model at 0x7fd38a1e6200>, verbose=0),
             param_grid={'batch_size': [32, 64], 'epochs': [10, 20]})

In [8]:
# Obtener los mejores hiperparámetros
print(f"Mejores hiperparámetros: {grid.best_params_}")
print(f"Mejor puntuación: {grid.best_score_}")

Mejores hiperparámetros: {'batch_size': 64, 'epochs': 20}
Mejor puntuación: 0.531
